* 25 % carga y limpieza
    * Carga con Pandas: pd.read_csv

    * Limpieza de valores error: 
        * hay columnas que tienen un valor '?', por tanto se deben reemplazar por un valor nan.

    * Cambio de tipo de dato: .astype() a numéricos y textos, por ejemplo a categorical y carat a float32 o float16

    * Limpieza de nulos (limpiar valores NaN):
        * Nulos en columnas continuas: mediana, media
        * Nulos en columnas categóricas: moda, un valor fijo

    * Encoding: texto a numérico
        * Uso de la función get_dummies() para encoding one_hot
        * Uso de map para encoding ordinal para la columna cut: 1, 2, 3, 4

* 25 % transformaciones:
    * Uso de función apply:
        * crear una columna price_iva a partir de la columna price que muestre el precio + IVA (21%)

    * Crear una nueva columna volumen combinando: x * y * z

    * Ordenar por dos columnas con sort_values():
        * tipo de corte (cut) y precio (price)

    * Agrupaciones con groupby y visualizarla
        * Agrupar por las 3 que hay de tipo categórico calculando la media, max, min por ejemplo de alguna de las numéricas: price, carat, depth

20 % distribuciones: 

* Outliers: Visualización Q1 y Q3 y calcular límites tukey y filtrar. Sobre la columna precio.

* asimetría, curtosis y transformar datos con logaritmo o raíz cuadrada

* Discretizar la columna precio por barato, medio, caro usando la función pd.cut

30 % visualizaciones:

* Seaborn EDAS:
    * univariantes:
        * histogramas y curvas de densidad
        * boxplot
        * countplot
    * bivariantes y multivariantes
        * scatterplot con hue, con size, con style
        * Calcular correlación con Pandas y mostrarla con seaborn
        * Hacer la correlación en un gráfico de barras para la columna 'price'
        * Crear una pivot table usando como index y columns algunas variables categóricas y como values usar el price y visualizarla con heatmap de seaborn
    * Combinarlas con:
        * hue, style, size, row, col usando un relplot
        * filtro


### Librerías a utilizar

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carga y Limpieza
### Carga csv con pandas

In [ ]:
df = pd.read_csv("diamonds.csv")
df.head(2)

In [ ]:
df.info()


### Limpieza de valores error

In [4]:
df = df.replace('?', np.nan)

### Cambio de tipo de datos

In [5]:
df["carat"] = df["carat"].astype("float32")
df["price"] = df["price"].astype("float")
df["x"] = df["x"].astype("float32")
df["clarity"] = df["clarity"].astype("category")

In [ ]:
df.info()

### Limpieza NaN

In [ ]:
df.count()

In [ ]:
df.isna().sum()

In [9]:
df["carat"] = df["carat"].fillna(df["carat"].mean())
df["cut"] = df["cut"].fillna(df["cut"].mode().iloc[0])
df["color"] = df["color"].fillna(df["color"].mode().iloc[0])
df["clarity"] = df["clarity"].fillna(df["clarity"].mode().iloc[0])
df["table"] = df["table"].fillna(df["table"].mean())
df["price"] = df["price"].fillna(df["price"].mean())
df["price"] = df["price"].astype("int32")
df["x"] = df["x"].fillna(df["x"].mean())
df["y"] = df["y"].fillna(df["y"].mean())
df["z"] = df["z"].fillna(df["z"].mean())

In [ ]:
df.info()


## Encoding

* Get_dummies()

In [ ]:
df_encoded = pd.get_dummies(df)
df_encoded

* Encodig Ordinal

In [ ]:
df["cut"].value_counts()

In [ ]:
mapeo_cut = {
    "Ideal": 1,
    "Premium": 2,
    "Very Good": 3,
    "Good": 4,
    "Fair": 5
}

df["cut"] = df["cut"].map(mapeo_cut)
df["cut"] = df["cut"].astype(int)
df.head()

### Creacion columnas

* price_iva

In [ ]:
df["price_iva"] = df["price"] * 1.21
df.head()

* volumen ( x, y, z)

In [ ]:
df["volume"] = df["x"] * df["y"] * df["z"]
df.head()

### Orden de columnas


* Tipo de corte

In [ ]:
df.sort_values("cut")


* Precio

In [ ]:
df.sort_values("price", ascending=False)
df.head()

### Agrupaciones group_by()

* carac

In [ ]:
df.groupby(by = "color").aggregate({"price":["min", "max", "mean"]})


* price

In [ ]:
df.groupby(by="cut", as_index = False).aggregate({"price":["min", "max", "mean"]})

In [ ]:
df. groupby('cut')[['carat', 'depth', 'table', 'price']]. agg(['mean', 'min', 'max']) 

* depth

In [21]:
#df.groupby(by= "depth", as_index = False)

## Seaborn EDAS
### Univariantes

* Histograma y curva de densidad

In [ ]:
sns.histplot(df, x="price", kde=True, color="green", hue="cut", bins=100)
plt.axvline(df["price"].mean(), color="magenta", linewidth=2)

In [ ]:
sns.kdeplot(df, x="price")
sns.rugplot(df, x="price")
plt.axvline(df["price"].mean(), color="magenta")
plt.axvline(df['price'].quantile(0.25), color='magenta', linestyle='--')
plt.axvline(df['price'].quantile(0.75), color='magenta', linestyle='--')

* Boxplot

In [ ]:
sns.boxplot(df, x=df["price"], hue=df["cut"])

* Countplot

In [ ]:
sns.countplot(df, x="price")

### Bivariantes y Multivariantes
* Scatterplot

In [ ]:
sns.scatterplot(df, x="price", y="depth", hue="color")


In [ ]:
sns.scatterplot(df, x="price", y="depth", hue="color", style="cut")


In [ ]:
sns.scatterplot(df, x="volume", y="price", hue="color", style="cut", size="depth")

In [ ]:
sns.jointplot(df, x="price", y="depth", hue="color")

#### Correlacion
* Calculo con pandas

In [ ]:
correlacion = df_encoded.corr()
correlacion

* Mostrar con Seaborn

In [ ]:
sns.heatmap(correlacion, annot=True, cmap="viridis")

In [ ]:
sns.heatmap(correlacion[["price", "x", "y", "z", "carat"]].corr(), annot=True, cmap="viridis") 

* Combinarlas con filtros hue, style

In [33]:
#

## Outliers
* Visualizacion Q1 y Q3 

In [34]:
#

* Limites Tukey y filtro

In [35]:
#

## Transformacion de datos
* Asimetria

* Curtosis

* Transformacion logaritmo

* Transformacion raiz cuadrada

## Discretizar la columna precio
* barato, medio, caro